In [ ]:
import os
import io
from google.cloud import videointelligence

In [ ]:
from IPython.display import Video
filename = #file path
Video(filename
     ,embed=True)

In [ ]:
class video_annotation:  
    def __init__(self, vld, lcd,seg,conf):  
        self.vld = vld  
        self.lcd = lcd
        self.seg = seg  
        self.conf = conf

In [ ]:
def write_file(segment_level_annotation,shot_level_annotation,frame_level_annotation,filename):
    annotations = segment_level_annotation+shot_level_annotation+frame_level_annotation
    filename = filename+'.txt'
    with open(filename, 'w') as f:
            for item in annotations:
                f.write(item.vld+"\n")
                f.write(item.lcd+"\n")
                f.write(item.seg+"\n")
                f.write(item.conf+"\n")

In [ ]:
try:
    """Detect labels given a file path."""
    video_client = videointelligence.VideoIntelligenceServiceClient()
    features = [videointelligence.Feature.LABEL_DETECTION]
    # filename_path - please mention your local file path
    filename = ''
    filename_path = ''+filename+'.mp4'
    segment_level_annotation = []
    shot_level_annotation = []
    frame_level_annotation = []
    with io.open(filename_path, "rb") as movie:
        input_content = movie.read()
    operation = video_client.annotate_video(
        request={"features": features, "input_content": input_content}
    )
    print("\nProcessing video for label annotations:")

    result = operation.result(timeout=90)
    print("\nFinished processing.")
    # Process video/segment level label annotations
    segment_labels = result.annotation_results[0].segment_label_annotations
    for i, segment_label in enumerate(segment_labels):
#         print("Video label description: {}".format(segment_label.entity.description))
        for category_entity in segment_label.category_entities:
            pass
#             print(
#                 "\tLabel category description: {}".format(category_entity.description)
#             )

        for i, segment in enumerate(segment_label.segments):
            start_time = (
                segment.segment.start_time_offset.seconds
                + segment.segment.start_time_offset.microseconds / 1e6
            )
            end_time = (
                segment.segment.end_time_offset.seconds
                + segment.segment.end_time_offset.microseconds / 1e6
            )
            positions = "{}s to {}s".format(start_time, end_time)
            confidence = segment.confidence
#             print("\tSegment {}: {}".format(i, positions))
#             print("\tConfidence: {}".format(confidence))
            segment_level_annotation.append(video_annotation("Video label description: {}".format(segment_label.entity.description),
                                                            "\tLabel category description: {}".format(category_entity.description),
                                                            "\tSegment {}: {}".format(i, positions),
                                                            "\tConfidence: {}".format(confidence)))
#         print("\n")

    # Process shot level label annotations
    shot_labels = result.annotation_results[0].shot_label_annotations
    for i, shot_label in enumerate(shot_labels):
        """print("Shot label description: {}".format(shot_label.entity.description))"""
        for category_entity in shot_label.category_entities:
            pass
            """print(
                "\tLabel category description: {}".format(category_entity.description)
            )"""

        for i, shot in enumerate(shot_label.segments):
            start_time = (
                shot.segment.start_time_offset.seconds
                + shot.segment.start_time_offset.microseconds / 1e6
            )
            end_time = (
                shot.segment.end_time_offset.seconds
                + shot.segment.end_time_offset.microseconds / 1e6
            )
            positions = "{}s to {}s".format(start_time, end_time)
            confidence = shot.confidence
            """print("\tSegment {}: {}".format(i, positions))"""
            """print("\tConfidence: {}".format(confidence))"""
            shot_level_annotation.append(video_annotation("Shot label description: {}".format(shot_label.entity.description),
                                                         "\tLabel category description: {}".format(category_entity.description),
                                                         "\tSegment {}: {}".format(i, positions),
                                                         "\tConfidence: {}".format(confidence)))
        """print("\n")"""

    # Process frame level label annotations
    frame_labels = result.annotation_results[0].frame_label_annotations
    for i, frame_label in enumerate(frame_labels):
#         print("Frame label description: {}".format(frame_label.entity.description))
        for category_entity in frame_label.category_entities:
            pass
#             print(
#                 "\tLabel category description: {}".format(category_entity.description)
#             )

        # Each frame_label_annotation has many frames,
        # here we print information only about the first frame.
        frame = frame_label.frames[0]
        time_offset = frame.time_offset.seconds + frame.time_offset.microseconds / 1e6
#         print("\tFirst frame time offset: {}s".format(time_offset))
#         print("\tFirst frame confidence: {}".format(frame.confidence))
        frame_level_annotation.append(video_annotation("Frame label description: {}".format(frame_label.entity.description),
                                                      "\tLabel category description: {}".format(category_entity.description),
                                                      "\tFirst frame time offset: {}s".format(time_offset),
                                                      "\tFirst frame confidence: {}".format(frame.confidence)))
#         print("\n")
    write_file(segment_level_annotation,shot_level_annotation,frame_level_annotation,filename)
    print('labels written to a file - ',filename)
except BaseException as e:
    print('error - ',e)